In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from PIL import Image as PImage
from IPython.display import Image as NbImage, display

from fastai.vision import *
from torchvision import transforms

from data import ImageDataset
import numpy as np

In [5]:
def embed(learner:Learner, img:Image):
    return learner.predict(img)[1]

def _embed_imgset(learner:Learner, imgset):
    return [embed(learner, img) for img in imgset]

def _find_nneighbours(learner:Learner, img:Image, embedlist, n=5):
    imgembed = embed(learner, img)
    scores = [torch.dot(imgembed-emb, imgembed-emb) for emb in embedlist]
    results = sorted(zip(scores, range(len(embedlist))))
    return list(zip(*results[:n]))[1]

def find_nneighbours(learner:Learner, img:Image, imgset, n=5):
    embedlist = _embed_dataset(learner, imgset)
    imgembed = embed(learner, img).cpu()
    scores = [F.cosine_similarity(imgembed, emb) for emb in embedlist]
    results = sorted(zip(scores, range(len(imgset))), reverse=True)
    return list(zip(*results[:n]))[1]

def show_nneighbours(learner:Learner, img:Image, imgset, n=5, embedlist=None):
    if embedlist is None:
        embedlist = _embed_dataset(learner, imgset)
    idxs = _find_nneighbours(learner, img, embedlist, n)
    images = [NbImage(filename=datum[0]) for datum in [imgset[i] for i in idxs]]
    display(*images)

In [ ]:
data_dir='data'
data_split='val'
classes = ('person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush')
n_epochs = 1
emsize = 32
batch_size = 2
margin = 3.

In [ ]:
# Parameters
n_epochs = 100
batch_size = 8
emsize = 128

In [ ]:
tfms = transforms.Compose([
    transforms.ToTensor(),
    Image,
    partial(crop_pad, size=512, padding_mode="zeros")
])
dataset = ImageDataset(
    f"{data_dir}/classy_coconut/val",
    classes,
    tfms=tfms
)
n_classes = len(classes)

print(len(dataset))

In [ ]:
learner = load_learner("siamese", "export.pkl").to_fp32()

In [ ]:
embedlist = _embed_dataset(learner, dataset)
# np.save("classy_coconut_embeddings_val.npz", torch.stack(embedlist).numpy())

In [ ]:
img = tfms(PImage.open("images/000000008021-head.jpg"))
show_nneighbours(learner, img, dataset, embedlist=embedlist)